In [1]:
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


<hr style="border-width:4px; border-style:solid; border-color:coral"></hr>

# MPI_Reduce

<hr style="border-width:4px; border-style:solid; border-color:coral"></hr>

In this example, we use `MPI_Reduce` to compute the mean of a set of random numbers.  The strategy is : 

* Rank 0 allocates an array of length N and initializes entries to uniformly distributed random numbers in [0,1]

* Rank 0 then sub-divides the array into `nprocs` segments and sends each segment to a rank. 

* Each rank computes the sum of the array entries it receives

* Then, calling `MPI_Reduce`, the sum from all ranks is collected at rank 0. 

* Rank 0 collects the total sum, and divides by $N$.   

We seed the random number generator with a fixed value so that we can expect the mean to be the same, regardless of how many processors we run the program on.   This value should be close to 

    mean = 0.5053434404086818

In [4]:
%%file reduce_demo.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

double random_number()
{
  return (double) rand() / (double) RAND_MAX ;
}

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank,nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    /* Length of large array */
    int N = 8192;
    int Nlocal = N/nprocs;

    double *x;
    int tag = 0;
    if (rank == 0)
    {
        /* Seed random number generater */
        srand(1234);
        
        /* Allocate memory to create data */
        x = malloc(N*sizeof(double));
        
        /* Initialize array */
        for(int i = 0; i < N; i++)
        {
            x[i] = random_number();
        }
        
        /* Distribute subdomains of array to processors */
        for(int p = 1; p < nprocs; p++)
        {
            int dest = p;
            MPI_Send(&x[p*Nlocal],Nlocal,MPI_DOUBLE,dest,tag,MPI_COMM_WORLD);
        }
    }
    else
    {
        int source = 0;
        x = malloc(Nlocal*sizeof(double));
        MPI_Recv(x,Nlocal,MPI_DOUBLE,source,tag,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
    }

    double sum = 0;
    for(int i = 0; i < Nlocal; i++)
    {
        sum += x[i];
    }
    printf("Rank %d : %.10f\n",rank,sum);

    int root = 0;
    double total_sum;
    MPI_Reduce(&sum,&total_sum,1,MPI_DOUBLE,MPI_SUM,root,MPI_COMM_WORLD);    
    double mean = total_sum/N;

    if (rank == 0)
    {
        printf("Total sum : %.10f\n",total_sum);
        printf("Mean is %.16f\n",mean);
    }

    free(x);
    
    MPI_Finalize();
}

Writing reduce_demo.c


In [5]:
%%bash

rm -rf reduce_demo

mpicc -o reduce_demo reduce_demo.c -lm

mpirun -n 2 reduce_demo

Rank 0 : 2064.2558083610
Rank 1 : 2075.5176554669
Total sum : 4139.7734638279
Mean is 0.5053434404086818
